In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict
import sys

In [2]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

# model = SwinTransformer3D(**cfg)

In [3]:
def get_model(num_classes,cfg, shape_of_input=(10,3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)
    x = backbone(inputs, training= False)
    print("backbone", x.shape)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [4]:
shape_of_input = (5, 3, 8, 112,112)
model = get_model(5, cfg, shape_of_input=shape_of_input)

X = tf.random.normal(shape_of_input)
y = tf.random.uniform((shape_of_input[0],1), 0, 5, tf.dtypes.int32)
model.summary()

2022-09-06 16:07:21.732861: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-06 16:07:21.732985: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

shape_of_input:  (5, 3, 8, 112, 112)
backbone (None, 768, 4, 4, 4)
5 768 {'type': 'CrossEntropyLoss'} avg 0.5 0.01 {'training': True}
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3, 8, 112, 112)]  0         
                                                                 
 swin_transformer3d (SwinTra  (None, 768, 4, 4, 4)     29694438  
 nsformer3D)                                                     
                                                                 
 i3d_head_tf (I3DHead_tf)    (None, 5)                 3845      
                                                                 
Total params: 29,698,283
Trainable params: 27,854,315
Non-trainable params: 1,843,968
_________________________________________________________________


In [5]:
# inputs = tf.keras.Input(shape_of_input[1:])
# model(inputs, training=True)

In [6]:
y1 = model.predict(X)
y1

2022-09-06 16:07:24.244296: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-06 16:07:25.965611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step


array([[0.19190046, 0.1740714 , 0.16585574, 0.2925058 , 0.17566659],
       [0.18793844, 0.17378731, 0.15073566, 0.3059562 , 0.18158248],
       [0.19891362, 0.17366555, 0.15798151, 0.28447986, 0.18495952],
       [0.20885931, 0.17561027, 0.15789083, 0.2781472 , 0.17949237],
       [0.18301955, 0.16816126, 0.15533166, 0.31540492, 0.17808262]],
      dtype=float32)

In [7]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(), metrics=["accuracy"])

model.fit(X, y, epochs=3)

Epoch 1/3


2022-09-06 16:07:33.448426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 14s 14s/step - loss: 1.5948 - accuracy: 0.2000
Epoch 2/3
1/1 [==============================] - 1s 895ms/step - loss: 1.5547 - accuracy: 0.4000
Epoch 3/3
1/1 [==============================] - 1s 620ms/step - loss: nan - accuracy: 0.0000e+00


In [8]:
y1 = model.predict(X)
y1

2022-09-06 16:07:45.181539: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step


array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]], dtype=float32)

In [9]:
y2 = model(X, training=False)
y2

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]], dtype=float32)>

In [10]:
model.layers[1].layers[5].weights

[<tf.Variable 'swin_transformer3d/basic_layer_3/swin_transformer_block3d_10/layer_normalization_23/gamma:0' shape=(768,) dtype=float32, numpy=
 array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, n